In [18]:
import numpy as np
import math
import pandas as pd
import sklearn as sk
import os
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns; sns.set_theme()

### Meta Data

In this project, we are predicting the probability that an online transaction is fraudulent, 
as denoted by the binary target isFraud. The data is broken into two files identity and transaction, 
which are joined by TransactionID. Not all transactions have corresponding identity information.

**Transaction Table**

- **TransactionDT**: timedelta from a given reference datetime (not an actual timestamp)
- **TransactionAmt**: transaction payment amount in USD
- **ProductCD**: product code, the product for each transaction
- **card1 - card6**: payment card information, such as card type, card category, issue bank, country, etc.
- **addr1, addr2**: “both addresses are for purchaser; addr1 as billing region; addr2 as billing country”
- **dist: distance**; "distances between (not limited) billing address, mailing address, zip code, IP address, phone area, etc.”
- **P_ and (R__) emaildomain**: purchaser and recipient email domain (certain transactions don't need recipient, so R_emaildomain is null.)
- **C1-C14**: counting, such as how many addresses are found to be associated with the payment card, etc. The actual meaning is masked. (maybe counts of phone numbers, email addresses, names associated with the user, device, ipaddr, billingaddr, etc. Also these are for both purchaser and recipient, which doubles the number.)
- **D1-D15**: timedelta, such as days between previous transaction, etc.
- **M1-M9**: match, such as names on card and address, etc.
- **Vxxx**: Vesta engineered rich features, including ranking, counting, and other entity relations.

**Identity Table**

- **DeviceType**
- **DeviceInfo**
- **id_01 - id_38**
- Variables in this table are identity information – network connection information (IP, ISP, Proxy, etc) and digital signature (UA/browser/os/version, etc) associated with transactions.
- They're collected by Vesta’s fraud protection system and digital security partners.
- The field names are masked and pairwise dictionary will not be provided for privacy protection and contract agreement

In [141]:
#TransactionID - No missing
#isFraud - No missing
#TransactionDT - Time Stamp Data (Will clean later)
#TransactionAmt - No missing
#ProductCD - No missing
#card1 - card6 - Card1: No missing, Card2: 8933, 1.5%, Card3: 1565, 0.26%, Card4: 1577, 0.27%, Card5: 4259, 0.72%, Card6: 1571, 0.27%
#addr1 - addr2 - addr1: 65706, 11.12%, addr2: 65706, 11.12%
#dist1 - dist2 - dist1: 352271, 59.65%, dist2: 552913, 93.62%
#P_emaildomain, R_emaildomain - P_emaildomain: 94456, 15.99%, R_emaildomain: 453249, 76.75%
#C1 - C14 - C1: No missing; C2: No missing; C3: No missing; C4: No missing; C5: No missing; C6: No missing; C7: No missing; C8: No missing; C9: No missing, C10: No missing, C11: No missing, C12: No missing, C13: No missing, C14: No missing;
#D1 - D15
#M1 - M9
#V1 - V339

**Timeline**

- 4/19: Transaction: Cards, addr, dist, email, 
- 4/21: Transaction: D, M
- 4/23: Transaction: V
- 4/25: Identity: Device Type, Device Info
- 4/27: Identity: id_01 - id_38
- 4/28: Done with Feature Selection and Engineering

- Check Column dependency
    - Categorical Columns
        - Group by missing or not, check final label distribution
        - Group by missing or not, check other columns distribution
        - Group by column labels, check other columns distribution

- Filling Missing values for quantitative columns 
    - Compute the correlation between other quantitative columns
    - Graph the scatterplot between the two columns
    - Impute with value prediction based on other columns with large correlation (>=75%)
    - Add a categorical column stating null or not

- Filling Missing values for categorical columns 
    - Add null as a column value
    - TBD
    
- One hot encode
- Standadization/normalization
    - Clip outlier values 

- PCA

### Load Data

In [19]:
#Data path
train_transaction_path = "data/train_transaction.csv"
train_identity_path = "data/train_identity.csv"
test_transaction_path = "data/test_transaction.csv"
test_identity_path = "data/test_identity.csv"
sample_submission_path = "data/sample_submission.csv"

#Read data
train_transaction = pd.read_csv(train_transaction_path)
train_identity = pd.read_csv(train_identity_path)
test_transaction = pd.read_csv(test_transaction_path)
test_identity = pd.read_csv(test_identity_path)
sample_submission = pd.read_csv(sample_submission_path)

train_df = pd.merge(train_identity, train_transaction, on="TransactionID", how='outer')
train_transaction_row = train_transaction.shape[0]
train_transaction_col = train_transaction.shape[1]
train_identity_row = train_identity.shape[0]
train_identity_col = train_identity.shape[1]
train_df_row = train_df.shape[0]
train_df_col = train_df.shape[1]
print('Transaction Training DataFrame: {} rows & {} columns.'.format(train_transaction_row, train_transaction_col))
print('Identity Training DataFrame: {} rows & {} columns.'.format(train_identity_row, train_identity_col))
print('Training DataFrame: {} rows & {} columns.'.format(train_df_row, train_df_col))

Transaction Training DataFrame: 590540 rows & 394 columns.
Identity Training DataFrame: 144233 rows & 41 columns.
Training DataFrame: 590540 rows & 434 columns.


### Categorical Data

In [20]:
identity_col = train_identity.columns
transaction_col = train_transaction.columns
# train_df[train_df.columns[train_df.dtypes==object]].head(2)
# train_transaction[train_transaction.columns[train_transaction.dtypes==object]].head(2)
# train_identity[train_identity.columns[train_identity.dtypes==object]].head(2)
cat_train_df = train_df[train_df.columns[train_df.dtypes==object]]

In [21]:
#returns a table with column name, unique value in each column, and the percentage of unique value in a column
def df_summary_cat(df):
    output = {'col_name': [], 'src': [], 'contain_nan': [], 'unique_val': [], 
              'unique_perc': [], 'null_val': [], 'null_perc': [], 
              'value': [], 'value_perc': [], 'most_common': [], 'redundent_perc': [], 
              'examples': []}

    for i in df.columns:
        unique_val = df[i].nunique()
        if i in identity_col:
            src = 'ID'
        else:
            src = 'TR'
        unique_val_with_nan = len(df[i].unique())
        contain_nan = True if unique_val != unique_val_with_nan else False
        unique_perc = round(unique_val/df.shape[0] * 100, 3)
        null_val = df[i].isnull().sum()
        null_perc = round(null_val / df.shape[0] * 100, 2)
        #The most common value / number of rows
        if len(df[i].value_counts()) > 0:
            redundent_perc = round(df[i].value_counts().iloc[0] / df.shape[0] * 100, 2)
        else:
            redundent_perc = 100
        
        value_val = df[i].describe()['count']
        value_perc = round(value_val / df.shape[0] * 100, 2)
        
        examples_func = lambda x : list(df[i].value_counts().index) if (x <= 4) else list(df[i].value_counts().index[:4])
        examples = examples_func(unique_val)
        output['col_name'].append(i)
        output['src'].append(src)
        output['contain_nan'].append(contain_nan)
        output['unique_val'].append(unique_val)
        output['unique_perc'].append(str(unique_perc) + '%')
        output['null_val'].append(null_val)
        output['null_perc'].append(str(null_perc) + '%')
        output['redundent_perc'].append(str(redundent_perc) + '%')
        output['value'].append(value_val)
        output['value_perc'].append(str(value_perc) + '%')
        output['most_common'].append(df[i].value_counts().index[0])
        output['examples'].append(examples)
    return pd.DataFrame(output)

In [23]:
cat_summary_df = df_summary_cat(cat_train_df)
cat_summary_df.head(5)

,col_name,src,contain_nan,unique_val,unique_perc,null_val,null_perc,value,value_perc,most_common,redundent_perc,examples
0,id_12,ID,True,2,0.0%,446307,75.58%,144233,24.42%,NotFound,20.83%,"[NotFound, Found]"
1,id_15,ID,True,3,0.001%,449555,76.13%,140985,23.87%,Found,11.47%,"[Found, New, Unknown]"
2,id_16,ID,True,2,0.0%,461200,78.1%,129340,21.9%,Found,11.23%,"[Found, NotFound]"
3,id_23,ID,True,3,0.001%,585371,99.12%,5169,0.88%,IP_PROXY:TRANSPARENT,0.59%,"[IP_PROXY:TRANSPARENT, IP_PROXY:ANONYMOUS, IP_..."
4,id_27,ID,True,2,0.0%,585371,99.12%,5169,0.88%,Found,0.87%,"[Found, NotFound]"


### Quantitative Data

In [24]:
quant_train_df = train_df[train_df.columns[train_df.dtypes!=object]]

In [25]:
def df_summary_quan(df):
    output = {'col_name': [], 'src': [], 'contain_nan': [], 'unique_val': [], 
              'unique_perc': [], 'null_val': [], 'null_perc': [], 'redundent_perc': [],
              'value': [], 'value_perc': [], 'mean': [], 'max': [], 'min': [], 'std': [], 
              '25%': [], '50%': [], '75%': [], 'examples': []}

    for i in df.columns:
        unique_val = df[i].nunique()
        if i in identity_col:
            src = 'ID'
        else:
            src = 'TR'
        unique_val_with_nan = len(df[i].unique())
        contain_nan = True if unique_val != unique_val_with_nan else False
        unique_perc = round(unique_val/df.shape[0] * 100, 3)
        null_val = df[i].isnull().sum()
        null_perc = round(null_val / df.shape[0] * 100, 3)
        if len(df[i].value_counts()) > 0:
            redundent_perc = round(df[i].value_counts().iloc[0] / df.shape[0] * 100, 3)
        else:
            redundent_perc = 100
        
        value_val = df[i].describe()['count']
        value_perc = round(value_val / df.shape[0] * 100, 2)
        mean_val = df[i].describe()['mean']
        std_val = df[i].describe()['std']
        min_val = df[i].describe()['min']
        twenty_five = df[i].describe()['25%']
        fifty = df[i].describe()['50%']
        seventy_five = df[i].describe()['75%']
        max_val = df[i].describe()['max']
        
        examples_func = lambda x : list(df[i].value_counts().index) if (x <= 4) else list(df[i].value_counts().index[:4])
        examples = examples_func(unique_val)
        output['col_name'].append(i)
        output['src'].append(src)
        output['unique_val'].append(unique_val)
        output['contain_nan'].append(contain_nan)
        output['unique_perc'].append(unique_perc)
        output['null_val'].append(null_val)
        output['null_perc'].append(null_perc)
        output['redundent_perc'].append(redundent_perc)
        output['value'].append(value_val)
        output['value_perc'].append(value_perc)
        output['mean'].append(round(mean_val, 2))        
        output['max'].append(max_val)        
        output['min'].append(min_val)
        output['std'].append(std_val)
        output['25%'].append(twenty_five)
        output['50%'].append(fifty)
        output['75%'].append(seventy_five)
        output['examples'].append(examples)
    return pd.DataFrame(output)

In [27]:
quan_summary_df = df_summary_quan(quant_train_df).sort_values('null_perc', ascending=False)
quan_summary_df.head()
quan_cols = ['null_perc', 'redundent_perc']
# df_summary_quan(d_data).sort_values('null_perc', ascending=False)

### Filter Columns

In [28]:
# #returns a table with column name, unique value in each column, and the percentage of unique value in a column
# def df_summary(df):
#     output = {'col_name': [], 'src': [], 'dtype' : [], 'unique_val': [], 'contain_nan': [],
#               'unique_perc': [], 'null_val': [], 'null_perc': [], 'redundent_perc': [],
#               'count': [], 'mean': [], 'max': [], 'min': [], 'std': [], 
#               '25%': [], '50%': [], '75%': [], 'examples': []}

#     for i in df.columns:
#         unique_val = df[i].nunique()
#         if i in identity_col:
#             src = 'ID'
#         else:
#             src = 'TR'
#         unique_val_with_nan = len(df[i].unique())
#         contain_nan = True if unique_val != unique_val_with_nan else False
#         unique_perc = round(unique_val/df.shape[0] * 100, 3)
#         null_val = df[i].isnull().sum()
#         null_perc = round(null_val / df.shape[0] * 100, 3)
#         if len(df[i].value_counts()) > 0:
#             redundent_perc = round(df[i].value_counts().iloc[0] / df.shape[0] * 100, 3)
#         else:
#             redundent_perc = 100
        
#         if df[i].dtype == 'float64':
#             count_val = df[i].describe()['count']
#             mean_val = df[i].describe()['mean']
#             std_val = df[i].describe()['std']
#             min_val = df[i].describe()['min']
#             twenty_five = df[i].describe()['25%']
#             fifty = df[i].describe()['50%']
#             seventy_five = df[i].describe()['75%']
#             max_val = df[i].describe()['max']
#         else:
#             count_val = np.nan
#             mean_val = np.nan
#             std_val = np.nan
#             min_val = np.nan
#             twenty_five = np.nan
#             fifty = np.nan
#             seventy_five = np.nan
#             max_val = np.nan
        
#         examples_func = lambda x : list(df[i].value_counts().index) if (x <= 4) else list(df[i].value_counts().index[:4])
#         examples = examples_func(unique_val)
#         output['col_name'].append(i)
#         output['src'].append(src)
#         output['dtype'].append(df[i].dtype)
#         output['unique_val'].append(unique_val)
#         output['contain_nan'].append(contain_nan)
#         output['unique_perc'].append(unique_perc)
#         output['null_val'].append(null_val)
#         output['null_perc'].append(null_perc)
#         output['redundent_perc'].append(redundent_perc)
#         output['count'].append(count_val)
#         output['mean'].append(mean_val)        
#         output['max'].append(max_val)
#         output['min'].append(min_val)
#         output['std'].append(std_val)
#         output['25%'].append(twenty_five)
#         output['50%'].append(fifty)
#         output['75%'].append(seventy_five)
#         output['examples'].append(examples)
#     return pd.DataFrame(output)

In [29]:
# summary_df = df_summary(train_df)
# summary_df.head(5)

In [31]:
# #Drop columns contains > 90% null observations
# print("Null Value Percent > 90%: {} Columns".format(summary_df[(summary_df['null_perc']>90)].shape[0]))
# #Drop columns contains > 90% redundent information
# print("Redundent Percent > 90%: {} Columns".format(summary_df[(summary_df['redundent_perc']>90)].shape[0]))
# #Drop columns contains > 90% null observations
# #Drop columns contains > 90% redundent information

# summary_df_clean = summary_df[(summary_df['null_perc']<=90) & (summary_df['redundent_perc']<=90)]
# summary_df_clean.sort_values('null_perc', ascending=False)
# summary_df[summary_df['redundent_perc']>=90] #redundent_perc > 90 && unique value < ???
# summary_df[summary_df['unique_val']<2]
# train_identity_box_df = train_identity.drop(columns=['TransactionID', 'id_02'])
# train_identity.head(2)
# train_transaction.head(2)

In [32]:
# train_transaction[['D1']].hist(bins=100)
# uniform_data = np.random.rand(10, 12)



# c_data = train_transaction[['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14']]
# c_pear_corr=c_data.corr(method='pearson')
# c_pear_corr

# d_data = train_transaction[['D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15']]
# d_pear_corr=d_data.corr(method='pearson')
# d_pear_corr

# fig, ax = plt.subplots(figsize=(8,8))
# im = ax.imshow(c_pear_corr, interpolation='nearest')
# fig.colorbar(im, orientation='vertical', fraction = 0.05)



# fig, ax = plt.subplots(figsize=(8,8))
# im = ax.imshow(d_pear_corr, interpolation='nearest')
# fig.colorbar(im, orientation='vertical', fraction = 0.05)




# fig, ax = plt.subplots(figsize=(16,16))
# im = ax.imshow(pear_corr, interpolation='nearest')
# fig.colorbar(im, orientation='vertical', fraction = 0.05)

# # Show all ticks and label them with the dataframe column name
# ax.set_xticklabels(our_data.columns, rotation=65, fontsize=20)
# ax.set_yticklabels(our_data.columns, rotation=0, fontsize=20)

# # Loop over data dimensions and create text annotations
# for i in range(len(our_data.columns)-1):
#     for j in range(len(our_data.columns)-1):
#         text = ax.text(j, i, round(pear_corr.to_numpy()[i, j], 2),
#                        ha="center", va="center", color="black")

# plt.show()

In [150]:
# pd.set_option('display.max_rows', None)
# cat_tr = cat_summary_df[cat_summary_df['src']=='TR'][['col_name', 'null_val', 'null_perc', 'examples']]
# quan_tr = quan_summary_df[quan_summary_df['src']=='TR'][['col_name', 'null_val', 'null_perc', 'examples']]
# tr_summary = pd.concat([cat_tr, quan_tr])
# tr_summary.sort_values(by=['col_name'])

In [ ]:
#returns a table with column name, unique value in each column, and the percentage of unique value in a column
def df_summary_cat(df):
    output = {'col_name': [], 'src': [], 'dtype' : [], 'unique_val': [], 'contain_nan': [],
              'unique_perc': [], 'null_val': [], 'null_perc': [], 'redundent_perc': [],
              'count': [], 'mean': [], 'max': [], 'min': [], 'std': [], 
              '25%': [], '50%': [], '75%': [], 'examples': []}

    for i in df.columns:
        unique_val = df[i].nunique()
        if i in identity_col:
            src = 'ID'
        else:
            src = 'TR'
        unique_val_with_nan = len(df[i].unique())
        contain_nan = True if unique_val != unique_val_with_nan else False
        unique_perc = round(unique_val/df.shape[0] * 100, 3)
        null_val = df[i].isnull().sum()
        null_perc = round(null_val / df.shape[0] * 100, 3)
        if len(df[i].value_counts()) > 0:
            redundent_perc = round(df[i].value_counts().iloc[0] / df.shape[0] * 100, 3)
        else:
            redundent_perc = 100
        
        if df[i].dtype == 'float64':
            count_val = df[i].describe()['count']
            mean_val = df[i].describe()['mean']
            std_val = df[i].describe()['std']
            min_val = df[i].describe()['min']
            twenty_five = df[i].describe()['25%']
            fifty = df[i].describe()['50%']
            seventy_five = df[i].describe()['75%']
            max_val = df[i].describe()['max']
        else:
            count_val = np.nan
            mean_val = np.nan
            std_val = np.nan
            min_val = np.nan
            twenty_five = np.nan
            fifty = np.nan
            seventy_five = np.nan
            max_val = np.nan
        
        examples_func = lambda x : list(df[i].value_counts().index) if (x <= 4) else list(df[i].value_counts().index[:4])
        examples = examples_func(unique_val)
        output['col_name'].append(i)
        output['src'].append(src)
        output['dtype'].append(df[i].dtype)
        output['unique_val'].append(unique_val)
        output['contain_nan'].append(contain_nan)
        output['unique_perc'].append(unique_perc)
        output['null_val'].append(null_val)
        output['null_perc'].append(null_perc)
        output['redundent_perc'].append(redundent_perc)
        output['count'].append(count_val)
        output['mean'].append(mean_val)        
        output['max'].append(max_val)        
        output['min'].append(min_val)
        output['std'].append(std_val)
        output['25%'].append(twenty_five)
        output['50%'].append(fifty)
        output['75%'].append(seventy_five)
        output['examples'].append(examples)
    return pd.DataFrame(output)